Load data from csv files

In [ ]:
import pandas as pd
from IPython.display import display, HTML
pd.options.display.max_rows = 200
pd.options.display.float_format = '{:,.2f}'.format


regions = pd.read_csv('./data/top-regions.csv')
clubs = pd.read_csv('./data/top-clubs.csv', index_col='id')
dates = pd.read_csv(
	'./data/top-clubs-dates.csv',
	index_col='id',
	parse_dates=['date']
)

## Show the top regions from [Resident Advisors event page](https://www.residentadvisor.net/events)

In [ ]:
regions.style.hide_columns(['rank', 'region'])

## Group regions in the top 20 by country

In [ ]:
by_country = regions[['country', 'name']].groupby('country')
by_country.agg(['count']).sort_values(by=('name', 'count'), ascending=False)

## Join top clubs with regions and show top clubs per region


In [ ]:

regions_top_clubs = regions.join(
    clubs.reset_index().set_index('region'),
    on='region',
    lsuffix='_region',
    rsuffix='_club'
)

clubs_by_region = regions_top_clubs.pivot_table(
	values=['rank_region', 'rank_club'],
	index=['name_region', 'name_club']
).sort_values(by=['rank_region', 'rank_club'])
clubs_by_region.style.hide_columns(['rank_club', 'rank_region'])

Join top clubs and regions to club dates. Summarize results


In [ ]:
regions_top_club_dates = regions_top_clubs.join(
    dates.reset_index().set_index('club_id'),
    on = 'id',
    rsuffix = '_date'
)

## Regions by number of events and total people attending

In [ ]:
regions_top_club_dates.groupby('name_region').agg(
    {'id': 'count', 'attending': ['sum', 'mean']}
).sort_values(by=('id', 'count'), ascending=False).rename(columns={'id': 'Number of Events'})

## Number of events per year per region

In [ ]:
regions_top_club_dates.groupby(
	[regions_top_club_dates['date'].map(lambda x: x.year)]
).agg(
    {'id': 'count', 'attending': ['sum', 'mean']}
).sort_values(by=('id', 'count'), ascending=False).rename(columns={'id': 'Number of Events'})

## Average yearly club attendance

In [ ]:
regions_top_club_dates.groupby(
	['name_club']
).agg({'id': 'count', 'attending': 'mean'}
).sort_values(by='attending', ascending=False).rename(columns={'id': 'Number of Events'}).head(n=20)